In [1]:
import sys
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm
import librosa
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
import cv2
import torch
#import torchaudio

from tools import prepare, mixup, preprocess, noise, getCorrects

sys.path.append('../BAT/datasets/')

classes = {
    "Rhinolophus ferrumequinum": 0,
    "Rhinolophus hipposideros": 1,
    "Myotis daubentonii": 2,
    "Myotis brandtii": 3,
    "Myotis mystacinus": 4,
    "Myotis emarginatus": 5,
    "Myotis nattereri": 6,
    #"Myotis bechsteinii": 7,
    "Myotis myotis": 7,
    "Myotis dasycneme": 8,
    "Nyctalus noctula": 9,
    "Nyctalus leisleri": 10,
    "Pipistrellus pipistrellus": 11,
    "Pipistrellus nathusii": 12,
    "Pipistrellus kuhlii": 13,
    "Eptesicus serotinus": 14,
    "Eptesicus nilssonii": 15,
    #"Plecotus auritus": 16,
    #"Plecotus austriacus": 16,
    #"Barbastella barbastellus": 16,
    #"Tadarida teniotis": 16,
    "Miniopterus schreibersii": 16,
    #"Hypsugo savii": 18,
    "Vespertilio murinus": 17,
}

2023-01-12 17:43:28.905686: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-12 17:43:29.311717: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-12 17:43:32.207423: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ffundel/.local/lib:
2023-01-12 17:43:32.207749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

# Dataset

In [ ]:
nfft = 512
num_bands = nfft // 2 + 1

max_len = 60
patch_len = 44
patch_skip = 22

samples_per_step = patch_skip * (nfft // 4)
seq_len = (max_len + 1) * samples_per_step
seq_skip = (max_len + 1) * samples_per_step // 4
    
data_path = "../BAT/datasets/prepared_signal.h5"
X_train, Y_train, X_test, Y_test, X_val, Y_val = prepare(data_path, classes, seq_len, seq_skip)

In [ ]:
print("Total sequences:", len(X_train) + len(X_test) + len(X_val))
print("Train sequences:", X_train.shape, Y_train.shape)
print("Test sequences:", X_test.shape, Y_test.shape)
print("Validation sequences:", X_val.shape, Y_val.shape)

In [ ]:
# Holdout for unsupervised training
holdout = True
h_split = 0.9
h_train = int(len(X_train) * h_split)
if holdout:
    X_unlabeled = X_train[:h_train]
    Y_unlabeled = Y_train[:h_train]
    
    X_train = X_train[h_train:]
    Y_train = Y_train[h_train:]
    
    print("Total sequences:", len(X_train) + len(X_test) + len(X_val))
    print("Train sequences:", X_train.shape, Y_train.shape)
    print("Test sequences:", X_test.shape, Y_test.shape)
    print("Validation sequences:", X_val.shape, Y_val.shape)
    print("Total unlabeled sequences:", len(X_unlabeled))
    print("Unlabeled sequences:", X_unlabeled.shape)
else:
    X_unlabeled = X_train[:h_train]
    Y_unlabeled = Y_train[:h_train]

# Model

In [ ]:
import time
import datetime
import torch
import torch.nn as nn
import math
from torch.cuda.amp import autocast
from torch.utils.data import TensorDataset, DataLoader

from torchsummary import summary
from torchmetrics.functional import f1_score

from SAM import SAM
from ASL import AsymmetricLoss
from BigBAT import BigBAT

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model, device_ids=[0, 1])
    model_student = nn.DataParallel(model_student, device_ids=[0, 1])

In [ ]:
from torchsummary import summary

patch_embedding = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(3, 5), stride=(2, 3), padding=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),

            nn.Conv2d(16, 32, kernel_size=(3, 5), stride=(2, 3), padding=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    
            nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 3), padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
    
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        ).to(device)

patch_embedding2 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(3, 5), stride=(2, 3), padding=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),

            nn.Conv2d(16, 32, kernel_size=(3, 5), stride=(2, 3), padding=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    
            nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 3), padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
    
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        ).to(device)

summary(patch_embedding, (1, 44, 257))

In [ ]:
batch_size = 32
epochs = 23
lr = 0.0005
warmup_epochs = 5
d_model = 64

nhead = 2
dim_feedforward = 32
num_layers = 2
dropout = 0.3
classifier_dropout = 0.3
num_classes = len(list(classes))
    
model = BigBAT(
    max_len=max_len,
    patch_len=patch_len,
    patch_skip=patch_skip,
    d_model=d_model,
    num_classes=len(list(classes)),
    patch_embedding=patch_embedding,
    use_cls=False,
    nhead=nhead,
    dim_feedforward=dim_feedforward,
    num_layers=num_layers,
    dropout=dropout,
    classifier_dropout=classifier_dropout,
)
model_student = BigBAT(
    max_len=max_len,
    patch_len=patch_len,
    patch_skip=patch_skip,
    d_model=d_model,
    num_classes=len(list(classes)),
    patch_embedding=patch_embedding2,
    use_cls=False,
    nhead=nhead,
    dim_feedforward=dim_feedforward,
    num_layers=num_layers,
    dropout=dropout,
    classifier_dropout=classifier_dropout,
)
    
model.to(device)
model_student.to(device)
print(device)

In [ ]:
#freq_masking = torchaudio.transforms.FrequencyMasking(freq_mask_param=50)
#time_masking = torchaudio.transforms.TimeMasking(time_mask_param=50)

In [ ]:
train_len = batch_size * int(len(X_train) / batch_size)
test_len = batch_size * int(len(X_test) / batch_size)
val_len = batch_size * int(len(X_val) / batch_size)

train_data = TensorDataset(X_train[:train_len], Y_train[:train_len])
test_data = TensorDataset(X_test[:test_len], Y_test[:test_len])
val_data = TensorDataset(X_val[:val_len], Y_val[:val_len])

train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)

unlabeled_len = batch_size * int(len(X_unlabeled) / batch_size)
unlabeled_data = TensorDataset(X_unlabeled[:unlabeled_len], Y_unlabeled[:unlabeled_len])
unlabeled_loader = DataLoader(unlabeled_data, batch_size=batch_size)

In [ ]:
def plot_sequence(X, Y, k):
    plt.figure(figsize = (20, 2.5))
    x = X[k].cpu().detach().numpy()
    y = Y[k].cpu().detach().numpy()
    plt.imshow(np.rot90(x), interpolation='nearest', aspect='auto', cmap='inferno')
    plt.colorbar()
    label_list = []
    if(len(y.shape) > 0):
        for i in np.argwhere(y == 1)[:,0]:
            label_list.append(list(classes)[i])
        plt.title(", ".join(label_list))
    else:
        plt.title(list(classes)[y])
    print(x.shape)

k = np.random.randint(0, batch_size)
X1, Y1 = next(iter(train_loader))
X1, Y1 = X1.cuda(), Y1.cuda()
X1, Y1 = mixup(X1, Y1, min_seq=1, max_seq=3)
X1 = preprocess(X1)
X1 = noise(X1)
#X1 = freq_masking(X1)
#X1 = time_masking(X1)
        
plot_sequence(X1, Y1, k)

In [ ]:
criterion = AsymmetricLoss(gamma_neg=2, gamma_pos=1, clip=0)

#base_optimizer = torch.optim.SGD
#optimizer = SAM(model.parameters(), base_optimizer, lr=lr, momentum=0.9)

optimizer = torch.optim.SGD(model_student.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4, nesterov=True, dampening=0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=warmup_epochs)

min_val_loss = np.inf

torch.autograd.set_detect_anomaly(True)

In [ ]:
def smoothen(y, num_classes, l=.02):
    return y * (1.0 - l) + (y.sum(dim=1, keepdim=True) * l / num_classes).repeat(1, num_classes)

def getSpeciesMasks(species):
    masks = torch.eye(len(species))
    for i, s in enumerate(species):
        genus = s.split(' ')[0]
        for k, ss in enumerate(species):
            if ss.startswith(genus):
                masks[i, k] = 1
        masks[i, i] = 0
    return masks

def speciesSmoothing(Y, masks, l=.1):
    ny = torch.zeros_like(Y)
    for i, y in enumerate(Y):
        lbl = torch.nonzero(y)
        ny[i] = y * (1.0 - l) + masks[lbl].sum(0) * (y.sum() * l / max(masks[lbl].sum(), 1))
    return ny

test_y = torch.zeros(3, num_classes).to(device)
test_y[0, 4] = 1
test_y[0, 5] = 1
test_y[1, 11] = 1
test_y[2, 4] = 1
test_y[2, 11] = 1
test_y[2, 17] = 1
print(test_y)
masks = getSpeciesMasks(list(classes)).to(device)
test_smooth = smoothen(test_y, num_classes)
test_specsmooth = speciesSmoothing(test_y, masks)
print(test_smooth, test_smooth.sum())
print(test_specsmooth, test_specsmooth.sum())

In [ ]:
model.load_state_dict(torch.load('BigBAT-trained.pth'))
model_student.load_state_dict(torch.load('BigBAT-trained.pth'))

In [ ]:
import torchvision.transforms as T
from contrastive_learner.contrastive_learner import RandomApply

augment = nn.Sequential(
    #RandomApply(T.ColorJitter(0.8, 0.8, 0.8, 0.2), p=0.8),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    RandomApply(T.GaussianBlur((23, 23), (1.5, 1.5)), p=0.3),
    RandomApply(noise, p=0.5),
    T.RandomErasing()
    #T.RandomResizedCrop((1343, 257))
)

In [ ]:
lambda_u = 2.0

def train_epoch(model, model_student, epoch, criterion, optimizer, scheduler, dataloader, dataloader_u, device):
    model_student.train()
    model.eval()
        
    num_batches = len(dataloader_u)
    num_samples = 0
    
    running_loss = 0.0
    running_corrects = 0

    iter_u = iter(dataloader_u)
        
    # Normal training procedure
    for batch, (inputs, labels) in enumerate(tqdm(dataloader)):
        try:
            inputs_u, _ = next(iter_u)
        except StopIteration:
            iter_u = iter(dataloader_u)
            inputs_u, _ = next(iter_u)
        
        inputs_u = inputs_u.to(device)
        inputs_u = preprocess(inputs_u)
        
        # Forward Pass to get the pseudo labels
        with torch.no_grad():
            outputs_pseudo = model(inputs_u).detach()
            outputs_pseudo = torch.sigmoid(outputs_pseudo)
            max_probs, pseudo_labels = torch.max(outputs_pseudo, 1)
            pseudo_labels = torch.nn.functional.one_hot(pseudo_labels, num_classes)
            #unlabeled_loss = alpha_weight(step) * criterion(outputs_pseudo, pseudo_labels)
            #unlabeled_loss = criterion(outputs_pseudo, pseudo_labels)
            
            # filter images that the teacher has low confidences on
            mask = max_probs.ge(0.6)
            pseudo_labels = pseudo_labels[mask]
        
        inputs, labels = inputs.to(device), labels.to(device)
        inputs, labels = mixup(inputs, labels, min_seq=1, max_seq=3)
        inputs = preprocess(inputs)

        # Augment inputs_u and inputs
        inputs = augment(inputs)
        inputs_u = augment(inputs_u)
        
        outputs_u = model_student(inputs_u[mask])
        unlabeled_loss = criterion(outputs_u, pseudo_labels)
        
        outputs = model_student(inputs)
        labeled_loss = criterion(outputs, labels) # reduction : mean
        loss = labeled_loss + lambda_u * unlabeled_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer.first_step()
        #loss = criterion(outputs, labels) + alpha_weight(step) * criterion(outputs_unlabeled, pseudo_labels)
        #loss.backward()
        #optimizer.second_step()

        # Calculate Loss
        running_loss += loss.item() * inputs.size(0)
        running_corrects += getCorrects(outputs_u, pseudo_labels)
        num_samples += inputs.size(0)
    
        # Perform learning rate step
        #scheduler.step()
        scheduler.step(epoch + batch / num_batches)
            
    epoch_loss = running_loss / num_samples
    epoch_acc = running_corrects / num_samples
    return epoch_loss, epoch_acc

In [ ]:
def test_epoch(model, epoch, criterion, optimizer, dataloader, device):
    model.eval()
    
    num_batches = len(dataloader)
    num_samples = len(dataloader.dataset)
    
    with torch.no_grad():
        running_loss = 0.0
        running_corrects = 0

        for batch, (inputs, labels) in enumerate(tqdm(dataloader)):
            # Transfer Data to GPU if available
            inputs, labels = inputs.to(device), labels.to(device)
            inputs, labels = mixup(inputs, labels, min_seq=1, max_seq=3)
            inputs = preprocess(inputs)

            # Clear the gradients
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(inputs)

            # Compute Loss
            loss = criterion(outputs, labels)

            # Calculate Loss
            running_loss += loss.item() * inputs.size(0)
            running_corrects += getCorrects(outputs, labels)

        epoch_loss = running_loss / num_samples
        epoch_acc = running_corrects / num_samples
    
    return epoch_loss, epoch_acc

# Training

In [ ]:
import wandb

wandb_config = {
    "epochs": epochs,
    "lr": lr,
    "batch_size": batch_size,
    "warmup_epochs": warmup_epochs,
    "d_model": d_model,
    "nhead": nhead,
    "dim_feedforward": dim_feedforward,
    "num_layers": num_layers,
    "dropout": dropout,
    "classifier_dropout": classifier_dropout
}

wandb.init(project="BigBAT", entity="frankfundel", config=wandb_config)

In [ ]:
iterations = 5
for i in range(iterations):
    for epoch in range(epochs):
        end = time.time()
        print(f"==================== Starting at epoch {epoch} ====================", flush=True)

        train_loss, train_acc = train_epoch(model, model_student, epoch, criterion, optimizer, scheduler, train_loader,
                                            unlabeled_loader, device)
        print('Training loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc), flush=True)

        #val_loss, val_acc = test_epoch(model, epoch, criterion, optimizer, val_loader, device)
        #print('Validation loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc), flush=True)
        
        val_loss, val_acc = test_epoch(model_student, epoch, criterion, optimizer, val_loader, device)
        print('Validation loss student: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc), flush=True)

        wandb.log({
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
        })

        if min_val_loss > val_loss:
            print('val_loss decreased, saving model', flush=True)
            min_val_loss = val_loss

            # Saving State Dict
            torch.save(model.state_dict(), 'BigBAT-NoisyStudent.pth')

    if i != iterations - 1:
        print("student becomes the new teacher")
        model = model_student

        # create new student model, optimizer, scheduler etc. / change batch sizes accordingly depending on GPU Memory
        #batch_size = batch_sizes[i+2]
        model_student = BigBAT(
            max_len=max_len,
            patch_len=patch_len,
            patch_skip=patch_skip,
            d_model=d_model,
            num_classes=len(list(classes)),
            patch_embedding=nn.Sequential(
                nn.Conv2d(1, 16, kernel_size=(3, 5), stride=(2, 3), padding=3),
                nn.BatchNorm2d(16),
                nn.ReLU(),

                nn.Conv2d(16, 32, kernel_size=(3, 5), stride=(2, 3), padding=3),
                nn.BatchNorm2d(32),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

                nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 3), padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(),

                nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 3), padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU(),

                nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU(),
            ),
            use_cls=False,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            num_layers=num_layers,
            dropout=dropout,
            classifier_dropout=classifier_dropout,
        )
        model_student = model_student.to(device)
        model_student.load_state_dict(torch.load('BigBAT-trained.pth'))
        if device == 'cuda':
            model_student = torch.nn.DataParallel(model_student) 
        optimizer = torch.optim.SGD(model_student.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4, nesterov=True, dampening=0)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer=optimizer, T_0=warmup_epochs)

In [ ]:
# Load after training
model.load_state_dict(torch.load('BigBAT-NoisyStudent.pth'))

# Evaluation

In [ ]:
mixed_corrects = 0.0
repeats = 5

predictions = []
targets = []

for r in range(repeats):
    # iterate over test data
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        inputs, labels = mixup(inputs, labels, min_seq=1, max_seq=3)
        inputs = preprocess(inputs)

        output = model(inputs) # Feed Network
        predictions.extend(output.data.cpu().numpy())
        targets.extend(labels.data.cpu().numpy())
        mixed_corrects += getCorrects(output, labels)

In [ ]:
from sklearn.metrics import f1_score

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

mixed_test_acc = mixed_corrects / (repeats * len(test_data))
mixed_f1_micro = f1_score(sigmoid(np.asarray(predictions)) > 0.5, np.asarray(targets), average='micro')
mixed_f1_macro = f1_score(sigmoid(np.asarray(predictions)) > 0.5, np.asarray(targets), average='macro')

print("Mixed test acc:", mixed_test_acc)
print("Mixed f1 micro:", mixed_f1_micro)
print("Mixed f1 macro:", mixed_f1_macro)

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params, "params")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

Y_pred = []
Y_true = []
corrects = 0

model.eval()

# iterate over test data
for inputs, labels in tqdm(test_loader):
    inputs, labels = inputs.to(device), labels.to(device)
    inputs = preprocess(inputs)
    
    output = model(inputs) # Feed Network

    output = torch.argmax(output, 1).data.cpu().numpy()
    Y_pred.extend(output) # Save Prediction

    labels = torch.argmax(labels, 1).data.cpu().numpy()
    #labels = labels.data.cpu().numpy()
    Y_true.extend(labels) # Save Truth

In [ ]:
# Build confusion matrix
cf_matrix = confusion_matrix(Y_true, Y_pred, normalize="all") # normalize{‘true’, ‘pred’, ‘all’}, default=None
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=-1), index = [i for i in classes], columns = [i for i in classes])
plt.figure(figsize = (12, 7))
s = sn.heatmap(df_cm, annot=True, fmt='.2f')
s.set(xlabel='True', ylabel='Pred')
plt.savefig('BigBAT-NoisyStudent.png')

In [ ]:
corrects = np.equal(Y_pred, Y_true).sum()
single_test_acc = corrects / len(Y_pred)
single_f1 = f1_score(Y_true, Y_pred, average=None).mean()

print("Single test accuracy:", single_test_acc)
print("Single F1-score:", single_f1)

In [ ]:
wandb.log({
    "mixed_test_acc": mixed_test_acc,
    "mixed_f1_micro": mixed_f1_micro,
    "mixed_f1_macro": mixed_f1_macro,
    "single_test_acc": single_test_acc,
    "single_f1_micro": single_f1,
    "num_params": pytorch_total_params,
})

wandb.finish()